In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [9]:
len(words)

32033

In [10]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']= 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [11]:
block_size = 3
X, Y = [], []
for w in words[:5]:

    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [12]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [51]:
C = torch.randn((27, 2))

In [52]:
emb = C[X]

In [53]:
emb.shape

torch.Size([32, 3, 2])

In [54]:
W1 = torch.randn((6, 100))
B1 = torch.randn(100)

In [55]:
# torch.cat(torch.unbind(emb, 1), 1) innefecient tho
# or
# emb.view(32, 6)
# + B1 will broadcast to all the rows
# 32, 100
#  1, 100
h = torch.tanh(emb.view(-1, 6) @ W1 + B1)
h

tensor([[ 0.9750, -0.8584,  0.9998,  ..., -0.2027, -0.9672,  0.9582],
        [ 0.9319,  0.8819,  0.9991,  ..., -0.8752, -0.6037, -0.1554],
        [-0.3531, -0.9991,  0.1017,  ..., -0.6171, -0.7087, -0.8201],
        ...,
        [-0.5995, -0.9809,  0.4512,  ..., -0.9812, -0.3518,  0.9564],
        [-0.9652, -0.8528,  0.9907,  ...,  0.3387,  0.8448, -0.9928],
        [-0.7199, -0.9989,  0.9993,  ...,  0.7031, -0.3398, -0.8649]])

In [56]:
W2 = torch.randn((100, 27))
B2 = torch.randn(27)

In [60]:
logits = h @ W2 + B2

In [62]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims = True)

In [66]:
loss = - prob[torch.arange(32), Y].log().mean()
loss

tensor(18.6107)